In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
from keras.layers import LSTM, Dense, Dropout, Input
from keras.models import Sequential
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import ydf
from utilFunctions import feature_engineering, calculate_model_returns

In [ ]:
price_history = pd.read_csv('data/ETH_USD_11_20_2020-1_19_2021_historical_prices.csv', sep=';')
whales = pd.read_csv('data/transactions_aggregated_21_22.csv')
validators = pd.read_csv('data/validators_aggregated_21_22.csv')

current_data = pd.read_csv('data/transaction_aggregated_25.csv')
current_data = pd.read_csv('data/validators_aggregated_25.csv')

## Data Engineering

In [ ]:

Xt, Xe, yt, ye = feature_engineering(num_labels=3, validators=validators, whales=whales, do_test_train_split=True)

### Running Data Engineering for various model input format needs

In [ ]:
# For YDP (TF Decision Forest Library)
# Train
ydf_train = pd.DataFrame(
    data=Xt.reshape(Xt.shape[0], -1),
    columns=[f"feature_{i}" for i in range(Xt.shape[1] * Xt.shape[2])]
)
ydf_train['labels'] = yt.values

# Test
ydf_test = pd.DataFrame(
    data=Xe.reshape(Xe.shape[0], -1),
    columns=[f"feature_{i}" for i in range(Xe.shape[1] * Xe.shape[2])]
)
ydf_test['labels'] = ye.values

model = ydf.RandomForestLearner(
    label= "labels",
    task=ydf.Task.CLASSIFICATION,
    num_trees=1000, # Default 300
    max_depth=8, #Default 16
    min_examples=10, #Default 5
).train(ydf_train,)

# Evaluate the model
evaluation_train = model.evaluate(ydf_train)
evaluation_test = model.evaluate(ydf_test)

y_pred_test = model.predict(ydf_test)

print(f"Train Accuracy: {evaluation_train.accuracy}")
print(f"Test Accuracy: {evaluation_test.accuracy}")
print(f"Train Confusion Matrix")
print(evaluation_train.confusion_matrix)
print(f"Test Confusion Matrix")
print(evaluation_test.confusion_matrix)

In [ ]:


## Tensorflow models:

# model = LogisticRegression(max_iter=1000, random_state=42) ## 40% Test
# model = MLPClassifier([24, 24, 24], max_iter=10000, random_state=42, solver='adam', activation='logistic') ## 50% Test

# Fit the model on the training data
# model.fit(X_train, y_train)

# Predict on training and test data
# y_train_pred = model.predict(X_train)
# y_test_pred = model.predict(X_test)

# Calculate accuracy - FIXED to use windowed labels
# train_accuracy = accuracy_score(y_train_windowed, y_train_pred)
# test_accuracy = accuracy_score(y_test_windowed, y_test_pred)

# print(f"Train Accuracy: {train_accuracy}")
# print(f"Test Accuracy: {test_accuracy}")

# # Generate the confusion matrix - FIXED to use windowed labels
# conf_matrix = confusion_matrix(y_test_windowed, y_test_pred, labels=range(len(labels)))

# # Plot the heatmap
# plt.figure(figsize=(8, 6))
# sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels)
# plt.xlabel('Predicted Labels')
# plt.ylabel('True Labels')
# plt.title('Confusion Matrix Heatmap')
# plt.show()

## Historical Price prediction

In [ ]:
## Get a vector of predictions for calculating backtesting (model returns based on history)

# Calculate model returns
calculate_model_returns(predictions=predictions, price_deltas=price_deltas_test, num_labels=num_labels,invest_on_tie=False, investment_rate=0.8)


